In [1]:
import torch

# Создаем входной массив из двух изображений RGB 3*3
input_images = torch.tensor(
      [[[[0,  1,  2],
         [3,  4,  5],
         [6,  7,  8]],

        [[9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]],


       [[[27, 28, 29],
         [30, 31, 32],
         [33, 34, 35]],

        [[36, 37, 38],
         [39, 40, 41],
         [42, 43, 44]],

        [[45, 46, 47],
         [48, 49, 50],
         [51, 52, 53]]]])


def get_padding2d(input_images):
    padded_images = torch.nn.functional.pad(input_images,(1,1,1,1))# добавить нулей с четырех сторон каждого изображения
    return padded_images.float()


correct_padded_images = torch.tensor(
       [[[[0.,  0.,  0.,  0.,  0.],
          [0.,  0.,  1.,  2.,  0.],
          [0.,  3.,  4.,  5.,  0.],
          [0.,  6.,  7.,  8.,  0.],
          [0.,  0.,  0.,  0.,  0.]],

         [[0.,  0.,  0.,  0.,  0.],
          [0.,  9., 10., 11.,  0.],
          [0., 12., 13., 14.,  0.],
          [0., 15., 16., 17.,  0.],
          [0.,  0.,  0.,  0.,  0.]],

         [[0.,  0.,  0.,  0.,  0.],
          [0., 18., 19., 20.,  0.],
          [0., 21., 22., 23.,  0.],
          [0., 24., 25., 26.,  0.],
          [0.,  0.,  0.,  0.,  0.]]],


        [[[0.,  0.,  0.,  0.,  0.],
          [0., 27., 28., 29.,  0.],
          [0., 30., 31., 32.,  0.],
          [0., 33., 34., 35.,  0.],
          [0.,  0.,  0.,  0.,  0.]],

         [[0.,  0.,  0.,  0.,  0.],
          [0., 36., 37., 38.,  0.],
          [0., 39., 40., 41.,  0.],
          [0., 42., 43., 44.,  0.],
          [0.,  0.,  0.,  0.,  0.]],

         [[0.,  0.,  0.,  0.,  0.],
          [0., 45., 46., 47.,  0.],
          [0., 48., 49., 50.,  0.],
          [0., 51., 52., 53.,  0.],
          [0.,  0.,  0.,  0.,  0.]]]])


print(torch.allclose(get_padding2d(input_images), correct_padded_images))

C:\python\__pycache__\envs\dsfs\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True


In [43]:
kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

In [44]:
def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width

# Реализация свертки через цикл

In [161]:
res = [[[[] for _ in range(row)] for _ in range(slow)]for _ in range(batch_s)]

In [164]:
torch.tensor(res)

tensor([[[[1493., 1327.],
          [1358., 1282.]]],


        [[[3788., 3163.],
          [3113., 2848.]]]])

In [162]:
stri = 2
size = kernel.size()[3]
for i in range(batch_s):
    for j in range(slow):
        for t in range(row):
            for y in range(col):
                cl = 0
                if (y*stri or t*stri) <= correct_padded_images.size()[3] - size:
                    for l in range(kernel.size()[1]):
                        cl += (correct_padded_images[i][l][t*stri:size+stri*t,y*stri:size+stri*y] * kernel[j][l]).sum()
                
                    res[i][j][t].append(cl)
                else:
                    continue

# Реализация свертки через матрицы первый способ

In [45]:
batch_s,chanel,row,col = calc_out_shape(list(correct_padded_images.size()),1,3,2,0)

In [46]:
size_ten,size_ker = correct_padded_images.size()[2],kernel.size()[2]

In [47]:
t = torch.zeros(row*row,size_ten*size_ten*size_ker)
stride = 2
count = 0 
for j in range(size_ker):
    col_l,col_r,row_u,row_d = 0,size_ten-size_ker,0,size_ten-size_ker
    for i in range(row*row):
        if i%row==0 and i!=0:
            row_u += stride
            row_d -= stride
            col_l,col_r = 0,size_ten-size_ker
            t[i,j*size_ten*size_ten:size_ten*size_ten*(j+1)] = torch.flatten(torch.nn.functional.pad(kernel[0][j],(col_l,col_r,row_u,row_d)),0)
        else:
            t[i,j*size_ten*size_ten:size_ten*size_ten*(j+1)] = torch.flatten(torch.nn.functional.pad(kernel[0][j],(col_l,col_r,row_u,row_d)),0)
        col_l += stride
        col_r -= stride
        count += 1

In [54]:
(t @ correct_padded_images.view(2,-1).permute(1,0)).view(batch_s,chanel,row,col)


tensor([[[[1493., 3788.],
          [1327., 3163.]]],


        [[[1358., 3113.],
          [1282., 2848.]]]])

# Реализация свертки через  матрицы второй способ

In [55]:
batch_s,chanel,row,col = calc_out_shape(list(correct_padded_images.size()),1,3,2,0)
size_ten,size_ker,size_chan = correct_padded_images.size()[2],kernel.size()[2],correct_padded_images.size()[1]

In [56]:
conv_ker = torch.flatten(kernel,1)

In [57]:
torch_res = torch.zeros(size_chanel*size_ker*size_ker,row*row*batch_s)
strok = 0
stride = 2
for ten in range(batch_s):
    for i in range(size_chanel):
        for j in range(row*row):
            if j%row == 0 and j!=0:
                strok += stride
            torch_res[i*size_ker*size_ker:size_ker*size_ker*(i+1),j+(ten*row*row)]\
            = torch.flatten(correct_padded_images[ten][i][strok:size_ker+strok,j*stride-(strok*stride):(size_ker+stride*j)-(strok*stride)],0)
        strok = 0   

In [59]:
(conv_ker @ torch_res).transpose(1,0).view(batch_s,chanel,row,col)

tensor([[[[1493., 1327.],
          [1358., 1282.]]],


        [[[3788., 3163.],
          [3113., 2848.]]]])